In [1]:
import pandas as pd
import numpy as np

import sklearn

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker 
import seaborn as sn

import os
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency

In [2]:
orifile1 = 'data/dft-road-casualty-statistics-vehicle-2023.csv'
orifile2 = 'data/dft-road-casualty-statistics-collision-2023.csv'

In [3]:
vehicle = pd.read_csv(orifile1)
collision = pd.read_csv(orifile2)

/tmp/ipykernel_18477/3601886663.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicle = pd.read_csv(orifile1)
/tmp/ipykernel_18477/3601886663.py:2: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  collision = pd.read_csv(orifile2)


In [4]:
# Select the columns of interest for analysis
columns_of_interest = [
    'accident_index', 'sex_of_driver', 'age_of_driver','driver_imd_decile']
vehicle_data_selected = vehicle[columns_of_interest]

# Saving the selected data to a local CSV file
vehicle_data_selected.to_csv('data/Q3/vehicle_data_selected.csv', index=False)

In [5]:
columns_of_interest2 = ['accident_index', 'accident_severity']
collision_data_selected = collision[columns_of_interest2]

# Saving the selected data to a local CSV file
collision_data_selected.to_csv('data/Q3/collision_data_selected.csv', index=False)

In [6]:
file1 = 'data/Q3/vehicle_data_selected.csv'
file2 = 'data/Q3/collision_data_selected.csv'
output_path = 'data/Q3/merged_output.csv'

dtype_dict = {0: 'str'}

# 定义读取的块大小
chunk_size = 10000  # 根据内存大小调整

# 创建一个写入器，先写入表头
with open(output_path, 'w') as output_file:
    first_chunk = True  # 标记是否是第一个分块
    for chunk_csv1 in pd.read_csv(file1, dtype=dtype_dict,chunksize=chunk_size):
        # 读取 csv2
        chunk_csv2 = pd.read_csv(file2, dtype=dtype_dict)

        # 合并当前块与csv2
        merged_chunk = pd.merge(chunk_csv1, chunk_csv2, on='accident_index', how='inner')

        # 写入合并结果，保留表头
        merged_chunk.to_csv(output_file, index=False, header=first_chunk, mode='a')
        first_chunk = False  # 之后的分块不需要重复写入表头

print(f"合并完成，结果已保存到 {output_path}")

合并完成，结果已保存到 data/Q3/merged_output.csv


In [7]:
# 替换所有可能的占位符为 NaN
merged_output = pd.read_csv('data/Q3/merged_output.csv')
merged_output.replace(["", " ", -1], pd.NA, inplace=True)
filtered_data = merged_output[merged_output['sex_of_driver'] != 3]

print(filtered_data.isin([-1]).sum())  # 检查每列中是否仍然存在 -1

# 再次删除 NaN
cleaned_data = filtered_data.dropna()
cleaned_file_path = 'data/Q3/cleaned_data.csv'
cleaned_data.to_csv(cleaned_file_path, index=False)
cleaned_data.head()

accident_index       0
sex_of_driver        0
age_of_driver        0
driver_imd_decile    0
accident_severity    0
dtype: int64


,accident_index,sex_of_driver,age_of_driver,driver_imd_decile,accident_severity
0,2023010419171,1,61,3,3
1,2023010419183,1,54,6,3
2,2023010419183,1,25,3,3
3,2023010419183,1,42,4,3
4,2023010419189,1,50,5,3


In [8]:
# For demonstration, create a feature that categorizes age into broader groups
def categorize_age(age):
    if 0 <= age < 18:
        return '0-17'
    elif 18 <= age <= 29:
        return '18-29'
    elif 30 <= age <= 59:
        return '30-59'
    elif 60 <= age <= 69:
        return '60-69'
    else:
        return '70+'

# 添加新的年龄分类列
cleaned_data['age_category'] = cleaned_data['age_of_driver'].apply(categorize_age)

# 定义自定义的年龄类别顺序
age_category_order = ['0-17', '18-29', '30-59', '60-69', '70+']

# 按顺序分配编码
category_to_code = {category: code for code, category in enumerate(age_category_order)}

# 应用自定义编码
cleaned_data['age_category_encoded'] = cleaned_data['age_category'].map(category_to_code)

# 查看结果
print("自定义类别编码映射：")
print(category_to_code)

print("\nDataframe 信息：")
print(cleaned_data.info())

print("\n带有自定义编码的前几行数据：")
cleaned_data.head()

自定义类别编码映射：
{'0-17': 0, '18-29': 1, '30-59': 2, '60-69': 3, '70+': 4}

Dataframe 信息：
<class 'pandas.core.frame.DataFrame'>
Index: 142064 entries, 0 to 189812
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   accident_index        142064 non-null  object
 1   sex_of_driver         142064 non-null  int64 
 2   age_of_driver         142064 non-null  object
 3   driver_imd_decile     142064 non-null  object
 4   accident_severity     142064 non-null  int64 
 5   age_category          142064 non-null  object
 6   age_category_encoded  142064 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 8.7+ MB
None

带有自定义编码的前几行数据：


/tmp/ipykernel_18477/2379451277.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['age_category'] = cleaned_data['age_of_driver'].apply(categorize_age)
/tmp/ipykernel_18477/2379451277.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['age_category_encoded'] = cleaned_data['age_category'].map(category_to_code)


,accident_index,sex_of_driver,age_of_driver,driver_imd_decile,accident_severity,age_category,age_category_encoded
0,2023010419171,1,61,3,3,60-69,3
1,2023010419183,1,54,6,3,30-59,2
2,2023010419183,1,25,3,3,18-29,1
3,2023010419183,1,42,4,3,30-59,2
4,2023010419189,1,50,5,3,30-59,2


In [10]:
# 定义目标变量和独立变量
target_var = 'accident_severity'
independent_vars = ['sex_of_driver', 'age_category','driver_imd_decile']

# 存储结果
results = []

# 对每个独立变量进行卡方检验
for var in independent_vars:
    # 创建列联表
    contingency_table = pd.crosstab(cleaned_data[target_var], cleaned_data[var])
    
    # 执行卡方检验
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    
    # 保存结果
    results.append({
        'Variable': var,
        'Chi2': chi2,
        'P-value': p,
        'Degrees of Freedom': dof
    })

# 转换为 DataFrame 格式输出
results_df = pd.DataFrame(results)
print(results_df)

            Variable        Chi2        P-value  Degrees of Freedom
0      sex_of_driver  486.286176  2.536872e-106                   2
1       age_category  484.136094   1.779252e-99                   8
2  driver_imd_decile  137.206639   2.208688e-20                  18


In [11]:
results_df = results_df.apply(pd.to_numeric, errors='coerce')

# Cramér's V 计算函数
def cramers_v(chi2, n, dof):
    return np.sqrt(chi2 / (n * dof))

# 迭代计算每个变量的 Cramér's V
results_df["Cramers_V"] = results_df.apply(
    lambda row: cramers_v(row["Chi2"], n=10000, dof=row["Degrees of Freedom"]), axis=1
)

# 输出结果
print(results_df)

   Variable        Chi2        P-value  Degrees of Freedom  Cramers_V
0       NaN  486.286176  2.536872e-106                   2   0.155930
1       NaN  484.136094   1.779252e-99                   8   0.077793
2       NaN  137.206639   2.208688e-20                  18   0.027609


In [ ]:
subset = cleaned_data[['accident_severity','age_category']]
subset.info()

In [ ]:
# 针对整个数据集，分别计算每个年龄段与 accident_severity 的卡方值和 P 值

# 构建完整的列联表，包含所有年龄段和事故严重程度
contingency_table_full = pd.crosstab(subset['age_category'], subset['accident_severity'])

# 对完整列联表进行卡方检验
chi2, p, dof, expected = chi2_contingency(contingency_table_full)

# 计算每个单元格的标准化残差
residuals = (contingency_table_full - expected) / (expected**0.5)

# 输出整体卡方和P值，并展示残差表
results_overall = {
    "Chi2": chi2,
    "P-value": p,
    "Degrees_of_Freedom": dof
}

# 转换为 DataFrame
residuals_df = pd.DataFrame(residuals)
# 打印显著残差信息
print(residuals_df)

In [ ]:
# 找出标准化残差中显著的单元格
# 绝对值 > 2 的残差被认为是显著的
significant_residuals = residuals_df.abs() > 2

# 提取显著单元格的信息
significant_cells = []
for age_category in residuals_df.index:
    for severity in residuals_df.columns:
        if significant_residuals.loc[age_category, severity]:
            significant_cells.append({
                "Age_Category": age_category,
                "Accident_Severity": severity,
                "Residual": residuals_df.loc[age_category, severity],
                "Interpretation": (
                    "Higher than expected" if residuals_df.loc[age_category, severity] > 0 
                    else "Lower than expected"
                )
            })

# 转换为 DataFrame 格式以便展示
significant_df = pd.DataFrame(significant_cells)

# 展示结果
# 打印显著残差信息
print("Significant standardized residuals analysis:")
print(significant_df)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
# 创建映射字典的逆映射
encoded_to_category = {v: k for k, v in age_category_mapping.items()}

# 替换 residuals_df 的索引为实际类别名称
residuals_df_with_labels = residuals_df.rename(index=encoded_to_category)

# 创建热力图
plt.figure(figsize=(10, 6))
sn.heatmap(
    residuals_df_with_labels,
    annot=True,  # 显示数值
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    cbar_kws={"label": "Standardized Residuals"}
)
plt.title("Standardized Residuals by Age Category and Accident Severity", fontsize=14)
plt.xlabel("Accident Severity", fontsize=12)
plt.ylabel("Age Category", fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# 画图：Chi2值
plt.figure(figsize=(10, 6))
plt.bar(results_df["Variable"], results_df["Chi2"], alpha=0.7)
plt.title("Chi-Square Test Results (Chi2 Values)", fontsize=14)
plt.xlabel("Variable", fontsize=12)
plt.ylabel("Chi2 Value", fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 图 2: P-values 条形图（对数尺度表示）

plt.figure(figsize=(10, 6))
plt.bar(results_df["Variable"], -results_df["P-value"].apply(lambda x: x if x > 0 else 1e-300).apply('log10'), color="#AA767C", alpha=1.0)
plt.title("Chi-Square Test Results (log10(P-values))", fontsize=14)
plt.xlabel("Variable", fontsize=12)
plt.ylabel("log10(P-value)", fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
"""
# 将指定列转换为数字格式
columns_to_convert = ['accident_severity', 'vehicle_type', 'sex_of_driver', 'age_of_driver', 'driver_imd_decile']  # 替换为你要转换的列名列表

for col in columns_to_convert:
    cleaned_data[col] = pd.to_numeric(cleaned_data[col], errors='coerce')
    cleaned_data = cleaned_data.dropna(subset=[col])  # 删除包含 NaN 的行
    cleaned_data[col] = cleaned_data[col].astype(int)  # 转换为整数
    cleaned_data = cleaned_data[~cleaned_data.isin([-1]).any(axis=1)]
print(cleaned_data.isin([-1]).sum())  # 检查每列中是否仍然存在 -1
# Display the summary of the selected columns to check their data types and missing values
cleaned_data
"""

In [ ]:
"""
# 绘制年龄段分布直方图
plt.figure(figsize=(6, 4))
ax = cleaned_vehicle['age_band_of_driver'].hist(
    bins=11, color='skyblue', edgecolor='black', alpha=0.7, label='Histogram')

ax.set_title("Number of Accidents by Driver's Age Band", fontsize=14, color='darkblue', pad=20)
ax.set_xlabel("Age Band of Driver", fontsize=12, color='black')
ax.set_ylabel("Number of Accidents", fontsize=12, color='black')
ax.xaxis.set_major_locator(plt.MultipleLocator(1))  # 设置 x 轴刻度每 1 显示一次
#ax.grid(axis='y', linestyle='--', alpha=0.7)  # 添加水平网格线

# 移除网格线
ax.grid(False)

# 添加图例
plt.legend(fontsize=10)

# 调整布局并显示图表
plt.tight_layout()
plt.show()
"""

In [ ]:
"""
# 绘制年龄分布直方图
plt.figure(figsize=(6, 4))
ax = cleaned_vehicle['age_of_driver'].hist(
    bins=100, color='skyblue', edgecolor='black', alpha=0.7, label='Histogram')

ax.set_title("Number of Accidents by Driver's Age", fontsize=14, color='darkblue', pad=20)
ax.set_xlabel("Age Band of Driver", fontsize=12, color='black')
ax.set_ylabel("Number of Accidents", fontsize=12, color='black')
ax.xaxis.set_major_locator(plt.MultipleLocator(5))  # 设置 x 轴刻度每 1 显示一次
#ax.grid(axis='y', linestyle='--', alpha=0.7)  # 添加水平网格线

# 移除网格线
ax.grid(False)

# 添加图例
plt.legend(fontsize=10)

# 调整布局并显示图表
plt.tight_layout()
plt.show()
"""

In [ ]:
"""
sex_of_driver_counts = cleaned_vehicle['sex_of_driver'].value_counts()

# 绘制饼图
plt.figure(figsize=(6, 6))
plt.pie(sex_of_driver_counts, 
        labels=sex_of_driver_counts.index, 
        autopct='%1.1f%%', 
        startangle=140, 
        colors=['skyblue', 'lightcoral', 'lightgreen'])
plt.title('Proportion of Accidents by Driver\'s Sex', fontsize=14)
plt.show()
"""

In [ ]:
"""
# 绘制直方图
plt.figure(figsize=(6, 4))
ax = cleaned_vehicle['driver_imd_decile'].hist(bins=10, color='skyblue', edgecolor='black', alpha=0.7, label='Histogram')

# 计算每个IMD分位数的事故数量
imd_counts = cleaned_vehicle['driver_imd_decile'].value_counts().sort_index()

# 叠加折线图
plt.plot(imd_counts.index, imd_counts.values, color='darkblue', marker='o', linestyle='-', linewidth=2, label='Trend Line')
plt.title("Number of Accidents by Driver's IMD Decile", fontsize=14, color='darkblue', pad=20)
plt.xlabel("IMD Decile (1: Most Deprived, 10: Least Deprived)", fontsize=12, color='black')
plt.ylabel("Number of Accidents", fontsize=12, color='black')
plt.xticks(range(1, 11))  # 显示1-10的刻度
plt.grid(axis='y', linestyle='--', alpha=0.7)  # 添加网格线
plt.legend(fontsize=10)  # 添加图例

plt.tight_layout()
plt.show()


# 设置图表样式
sn.set_style("whitegrid")

plt.figure(figsize=(6, 4))
ax = sn.countplot(
    data=cleaned_vehicle, 
    y='vehicle_type', 
    order=cleaned_vehicle['vehicle_type'].value_counts().index, 
    palette="coolwarm"
)
# 计算总数
total = len(cleaned_vehicle)

# 添加百分比标签
for p in ax.patches:
    width = p.get_width()
    ax.text(
        width + total * 0.01,  # 微调位置
        p.get_y() + p.get_height() / 2., 
        f'{width / total:.1%}', 
        va="center", 
        fontsize=10, 
        color='black'
    )

# 设置标题和标签
plt.title('Accident Frequency by Vehicle Type', fontsize=14, color='darkblue', pad=20)
plt.ylabel('Vehicle Type', fontsize=12, color='black')
plt.xlabel('Frequency', fontsize=12, color='black')

# 调整网格线样式
ax.xaxis.grid(True, linestyle='--', alpha=0.7)
ax.yaxis.grid(False)  # 关闭 y 轴的网格线

# 优化图表布局
plt.tight_layout()

# 显示图表
plt.show()
"""